In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import csv
import seaborn as sb
from collections import OrderedDict
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, precision_score, recall_score
from transformers import BertTokenizer
import torch
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bertopic import BERTopic
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch
import transformers

2023-01-07 21:01:46.864689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 21:01:46.961028: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-07 21:01:46.961044: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-07 21:01:47.417984: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
train_data = pd.read_csv('newest_train_data', engine='python', encoding='utf-8', on_bad_lines='skip').fillna('')
train_data = train_data[pd.to_numeric(train_data['text_sentiment_score'], errors='coerce').notnull()]
#train_data = train_data.sample(n=1000).reset_index(drop=True)

test_data = pd.read_csv('newest_test_data', engine='python', encoding='utf-8', on_bad_lines='skip').fillna('')
#test_data = test_data.sample(n=1000).reset_index(drop=True)

#test_data

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [5]:
def tokenize_data(data):
    labels = []
    topics = []
    input_ids = []
    attention_masks = []
    sentiment_score = []
    for index, line in data.iterrows():
        labels.append(int(line.label))
        topics.append(int(line.topic))
        sentiment_score.append(float(line.text_sentiment_score))
        encoded_dict = tokenizer.encode_plus(
                            line.idf_words,
                            add_special_tokens = True, # [CLS] & [SEP]
                            truncation = 'longest_first', # Control truncation
                            max_length = 100, # Max length about texts
                            pad_to_max_length = True, # Pad and truncate about sentences
                            return_attention_mask = True, # Attention masks
                            return_tensors = 'pt') # Return to pytorch tensors
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    try:
        labels = torch.tensor(labels)
    except:
        print(labels)

    topics = torch.tensor(topics)
    sentiment_score = torch.tensor(sentiment_score)
    return input_ids, attention_masks, labels, topics, sentiment_score

In [6]:
test_input_ids, test_attention_masks, test_labels, test_topics, test_sentiment_score = tokenize_data(test_data)
train_input_ids, train_attention_masks, train_labels, train_topics, train_sentiment_score = tokenize_data(train_data)

In [7]:
# Declare GPU as device to run on
device = torch.device("cuda")

# Move tensors to GPU
test_input_ids = test_input_ids.to(device)
test_attention_masks = test_attention_masks.to(device)
test_labels = test_labels.to(device)
test_topics = test_topics.to(device)
test_sentiment_score = test_sentiment_score.to(device)

train_input_ids = train_input_ids.to(device)
train_attention_masks = train_attention_masks.to(device)
train_labels = train_labels.to(device)
train_topics = train_topics.to(device)
train_sentiment_score = train_sentiment_score.to(device)


train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels, train_topics,train_sentiment_score)

val_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels, test_topics, test_sentiment_score)
len(train_dataset), len(val_dataset)

(57705, 14427)

In [8]:
device

device(type='cuda')

In [9]:
batch_size = 8
#num_workers = 4

train_dataloader = DataLoader(
                dataset=train_dataset,
                batch_size=batch_size,
                shuffle=True,
                drop_last=True)

val_dataloader = DataLoader(
                dataset=val_dataset,
                batch_size=batch_size,
                shuffle=True,
                drop_last=True)

BERT + TOPIC Model

In [10]:
class BertPlusModel(torch.nn.Module):
    def __init__(self, bert_model):
        super(BertPlusModel, self).__init__()
        self.bert_model = bert_model
        self.linear = torch.nn.Linear(769, 1)  # The output of BERT has size 768, so the input size to the linear layer should be 768
        self.sigmoid = torch.nn.Sigmoid()  # The sigmoid function will map the output to the range [0, 1]

    def forward(self, input_ids, attention_mask, additional_input):
        _, pooled_output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask).to_tuple()
        concatenated = torch.cat((pooled_output, additional_input.unsqueeze(dim=1)), dim=1)
        return self.linear(concatenated)

# Get the pretrained BERT model
bert_model = transformers.BertModel.from_pretrained('bert-base-uncased')

# Initialize the model with the pretrained BERT model
model = BertPlusModel(bert_model)

# Load model instead of training
#model.to('cuda')
#model.load_state_dict(torch.load('model.pt', map_location='cuda'))

# Move model to GPU
model = model.to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00004)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
model.train()  
total_step = len(train_dataloader.dataset)
for epoch in range(10):
    total_loss = 0
    for i, batch in enumerate(train_dataloader := tqdm(train_dataloader, total=len(train_dataloader))):
        # Unpack the input and label tensors from the batch
        input_ids, attention_masks, labels, topics, sentiment_scores = batch
        
        output = model(input_ids, attention_masks, topics)
        
        
        
        labels = labels.type(torch.float).unsqueeze(1)

        # Calculate the loss between the output and the labels
        loss = loss_fn(output, labels)
        
        # Backpropagate the loss and update the model's parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print('Epoch: {}, Loss: {:.4f}'.format(epoch+1, total_loss / total_step))

torch.save(model.state_dict(), "model_10epoch_8batch.pt")

100%|███████████████████████████████████████████████████████████████████████████████| 7213/7213 [30:08<00:00,  3.99it/s]


Epoch: 1, Loss: 0.0388


100%|███████████████████████████████████████████████████████████████████████████████| 7213/7213 [30:06<00:00,  3.99it/s]


Epoch: 2, Loss: 0.0868


100%|███████████████████████████████████████████████████████████████████████████████| 7213/7213 [30:01<00:00,  4.00it/s]


...

Epoch: 8, Loss: 0.0868


100%|███████████████████████████████████████████████████████████████████████████████| 7213/7213 [30:20<00:00,  3.96it/s]


Epoch: 9, Loss: 0.0868


100%|███████████████████████████████████████████████████████████████████████████████| 7213/7213 [29:41<00:00,  4.05it/s]


Epoch: 10, Loss: 0.0868


In [12]:
model.eval()

total_loss = 0
num_correct = 0
num_samples = 0
    
for epoch in range(10):
    for batch in val_dataloader:
            # Unpack the input and label tensors from the batch
            input_ids, attention_masks, labels, topics, sentiment_scores = batch
       
            with torch.no_grad():
                output = model(input_ids, attention_masks, topics)
                labels = labels.type(torch.float).unsqueeze(1)
                loss = loss_fn(output,  labels)
            
            # Calculate the loss between the output and the labels
            total_loss += loss.item() * input_ids.size(0)
            num_samples += input_ids.size(0)
            
            output = torch.round(torch.sigmoid(output))
            correct_entries = torch.eq(output, labels)
            num_correct += torch.sum(correct_entries)
        
    print(f'epoch: {epoch}, val_loss: {total_loss / num_samples:.4f}, val_acc: {num_correct / num_samples:.4f}')

epoch: 0, val_loss: 0.6921, val_acc: 0.5393
epoch: 1, val_loss: 0.6921, val_acc: 0.5393
epoch: 2, val_loss: 0.6921, val_acc: 0.5393
epoch: 3, val_loss: 0.6920, val_acc: 0.5393
epoch: 4, val_loss: 0.6920, val_acc: 0.5393
epoch: 5, val_loss: 0.6920, val_acc: 0.5393
epoch: 6, val_loss: 0.6921, val_acc: 0.5393
epoch: 7, val_loss: 0.6921, val_acc: 0.5393
epoch: 8, val_loss: 0.6920, val_acc: 0.5393
epoch: 9, val_loss: 0.6920, val_acc: 0.5393


In [13]:
def get_topic_id(article):
     for index, line in article.iterrows():
        return int(line.topic)

In [14]:
def filter_artciles_with_same_topic(article, dataset):
   topic = get_topic_id(article)
   filtered_articles = dataset.loc[dataset['topic'] == topic] 
   return filtered_articles
   

In [15]:
def filter_articles_based_on_predictions(articles, predictions, RECOMMENDATION_SIZE):
    result = []
    recommendations_found = 0
    for article_title, article_text, prediction in zip(articles['title'], articles['text'], predictions):
        if(prediction == 1):
            result.append([article_title, article_text])
            recommendations_found += 1
            if(recommendations_found >= RECOMMENDATION_SIZE):
                return result
    return result

In [16]:
def predict_label(query):
    query_input_ids, query_attention_masks, query_labels, query_topics, query_sentiment_score = tokenize_data(query)
    
    # Move tensors to GPU
    query_input_ids = query_input_ids.to(device)
    query_attention_masks = query_attention_masks.to(device)
    query_labels = query_labels.to(device)
    query_topics = query_topics.to(device)
    query_sentiment_score = query_sentiment_score.to(device)
    
    output = model(query_input_ids, query_attention_masks, query_topics)
    query_labels = query_labels.type(torch.float).unsqueeze(1)
    output = torch.round(torch.sigmoid(output))  
    return output


In [17]:
torch.cuda.empty_cache()

In [18]:
import gc
del test_input_ids
del test_attention_masks
del test_labels
del test_topics
del test_sentiment_score


del train_input_ids
del train_attention_masks
del train_labels
del train_topics
del train_sentiment_score

#test_input_ids = test_input_ids.to(device)
# test_attention_masks = test_attention_masks.to(device)
# test_labels = test_labels.to(device)
# test_topics = test_topics.to(device)
# test_sentiment_score = test_sentiment_score.to(device)

# train_input_ids = train_input_ids.to(device)
# train_attention_masks = train_attention_masks.to(device)
# train_labels = train_labels.to(device)
# train_topics = train_topics.to(device)
# train_sentiment_score = train_sentiment_score.to(device)
#


gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1796 MB |    2386 MB |  247491 GB |  247490 GB |\n|       from large pool |    1792 MB |    2357 MB |  239081 GB |  239080 GB |\n|       from small pool |       3 MB |      30 MB |    8409 GB |    8409 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    1796 MB |    2386 MB |  247491 GB |  247490 GB |\n|       from large pool |    1792 MB |    2357 MB |

In [19]:
RECOMMENDATION_SIZE = 5

query = test_data.sample(n=1).reset_index(drop=True)

print(get_topic_id(query))

articles_with_same_topic = filter_artciles_with_same_topic(query, test_data)
print(len(articles_with_same_topic))
predictions = predict_label(articles_with_same_topic).squeeze(1)
print(predictions)

recommended_articles = filter_articles_based_on_predictions(articles_with_same_topic, predictions, RECOMMENDATION_SIZE)

print(recommended_articles)

-1
5753


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.65 GiB (GPU 0; 8.00 GiB total capacity; 6.70 GiB already allocated; 0 bytes free; 6.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF